In [78]:
!pip install tensorflow opencv-python matplotlib pandas scikit-learn tqdm

In [79]:
import os
import tensorflow as tf
from PIL import Image
import pandas as pd
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from tensorflow.keras.utils import load_img , img_to_array
import numpy as np
from sklearn.metrics import confusion_matrix , classification_report
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [80]:
data_dir='data'
dt=pd.read_csv(os.path.join(data_dir,'biopsy.csv'))
dt.head()


,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [81]:
y = np.array(dt.get('diagnosis'))

print(y)
for i in range(len(y)):
    if y[i]=='B':
        y[i]=0
    else:
        y[i]=1
print(y)

['M' 'M' 'M' ... 'M' 'M' 'B']
[1 1 1 ... 1 1 0]


In [82]:
X= np.array(dt.drop('diagnosis',axis=1))
X.shape

(1138, 31)

In [83]:


def split_y(lista,porcentaje):
    tam_train= int(len(lista)*porcentaje)
    lista_train= lista[0:tam_train]
    lista_test= lista[tam_train:len(lista)]
    return (lista_train,lista_test)

X_train,X_test = split_y(X,0.80)
Y_train,Y_test = split_y(y,0.8)

In [84]:
X_train= np.array(X_train)
Y_train= np.array(Y_train)



In [92]:
X_train = np.asarray(X_train).astype(np.float32)
Y_train = np.asarray(Y_train).astype(np.float32)


In [96]:
print(X_train[:10])

[[842302.00 17.99 10.38 122.80 1001.00 0.12 0.28 0.30 0.15 0.24 0.08 1.10
  0.91 8.59 153.40 0.01 0.05 0.05 0.02 0.03 0.01 25.38 17.33 184.60
  2019.00 0.16 0.67 0.71 0.27 0.46 0.12]
 [842517.00 20.57 17.77 132.90 1326.00 0.08 0.08 0.09 0.07 0.18 0.06 0.54
  0.73 3.40 74.08 0.01 0.01 0.02 0.01 0.01 0.00 24.99 23.41 158.80
  1956.00 0.12 0.19 0.24 0.19 0.28 0.09]
 [84300904.00 19.69 21.25 130.00 1203.00 0.11 0.16 0.20 0.13 0.21 0.06
  0.75 0.79 4.59 94.03 0.01 0.04 0.04 0.02 0.02 0.00 23.57 25.53 152.50
  1709.00 0.14 0.42 0.45 0.24 0.36 0.09]
 [84348304.00 11.42 20.38 77.58 386.10 0.14 0.28 0.24 0.11 0.26 0.10 0.50
  1.16 3.44 27.23 0.01 0.07 0.06 0.02 0.06 0.01 14.91 26.50 98.87 567.70
  0.21 0.87 0.69 0.26 0.66 0.17]
 [84358400.00 20.29 14.34 135.10 1297.00 0.10 0.13 0.20 0.10 0.18 0.06
  0.76 0.78 5.44 94.44 0.01 0.02 0.06 0.02 0.02 0.01 22.54 16.67 152.20
  1575.00 0.14 0.20 0.40 0.16 0.24 0.08]
 [843786.00 12.45 15.70 82.57 477.10 0.13 0.17 0.16 0.08 0.21 0.08 0.33
  0.89 2.22 27.

In [97]:
ann = Sequential([
        
        Dense(8,input_dim=len(X_train[0,:]), activation='relu'),
        Dense(4, activation='relu'),
        Dense(1, activation='sigmoid')    
    ])

ann.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

ann.fit(X_train, Y_train, epochs=1000,verbose=1)

Epoch 1/1000
29/29 [==============================] - 0s 1ms/step - loss: 392873.4375 - accuracy: 0.4055
Epoch 2/1000
29/29 [==============================] - 0s 1ms/step - loss: 221327.2969 - accuracy: 0.4055
Epoch 3/1000
29/29 [==============================] - 0s 1ms/step - loss: 80101.8438 - accuracy: 0.4055
Epoch 4/1000
29/29 [==============================] - 0s 1ms/step - loss: 1414.2650 - accuracy: 0.5374
Epoch 5/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6841 - accuracy: 0.5945
Epoch 6/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6831 - accuracy: 0.5945
Epoch 7/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6821 - accuracy: 0.5945
Epoch 8/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6813 - accuracy: 0.5945
Epoch 9/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6805 - accuracy: 0.5945
Epoch 10/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6798 -

29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 82/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 83/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 84/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 85/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 86/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 87/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 88/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 89/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 90/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoc

29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 162/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 163/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 164/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 165/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 166/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 167/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 168/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 169/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 170/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.

29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 242/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 243/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 244/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 245/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 246/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 247/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 248/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 249/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 250/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.

29/29 [==============================] - 0s 2ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 322/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 323/1000
29/29 [==============================] - 0s 2ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 324/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 325/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 326/1000
29/29 [==============================] - 0s 2ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 327/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 328/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 329/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 330/1000
29/29 [==============================] - 0s 2ms/step - loss: 0.6752 - accuracy: 0.

...

29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 642/1000
29/29 [==============================] - 0s 2ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 643/1000
29/29 [==============================] - 0s 2ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 644/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 645/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 646/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 647/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 648/1000
29/29 [==============================] - 0s 2ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 649/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 650/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.

29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 722/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 723/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 724/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 725/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 726/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 727/1000
29/29 [==============================] - 0s 2ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 728/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 729/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 730/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.

29/29 [==============================] - 0s 998us/step - loss: 0.6752 - accuracy: 0.5945
Epoch 802/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 803/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 804/1000
29/29 [==============================] - 0s 2ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 805/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 806/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 807/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 808/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 809/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 810/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 

29/29 [==============================] - 0s 922us/step - loss: 0.6752 - accuracy: 0.5945
Epoch 882/1000
29/29 [==============================] - 0s 963us/step - loss: 0.6752 - accuracy: 0.5945
Epoch 883/1000
29/29 [==============================] - 0s 995us/step - loss: 0.6752 - accuracy: 0.5945
Epoch 884/1000
29/29 [==============================] - 0s 992us/step - loss: 0.6752 - accuracy: 0.5945
Epoch 885/1000
29/29 [==============================] - 0s 932us/step - loss: 0.6752 - accuracy: 0.5945
Epoch 886/1000
29/29 [==============================] - 0s 977us/step - loss: 0.6752 - accuracy: 0.5945
Epoch 887/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 888/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 889/1000
29/29 [==============================] - 0s 2ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 890/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - 

29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 962/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 963/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 964/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 965/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 966/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 967/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 968/1000
29/29 [==============================] - 0s 990us/step - loss: 0.6752 - accuracy: 0.5945
Epoch 969/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5945
Epoch 970/1000
29/29 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 

In [76]:
prediction= ann.predict(X_test)
print(prediction[:100])


8/8 [==============================] - 0s 3ms/step
[[0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.41120258]
 [0.4112025

In [75]:
print(Y_test[:5])

[0 0 1 0 0]


In [65]:
y_pred = ann.predict(X_test).round()
'''y_pred_classes = []
for element in y_pred:
    if(element>0.5):
        y_pred_classes.append(1)
    else:
        y_pred_classes.append(0)'''
    

print("Classification Report: \n", classification_report(Y_test, y_pred))

8/8 [==============================] - 0s 3ms/step


ValueError: Classification metrics can't handle a mix of unknown and binary targets